In [ ]:
import src.lightning as l

import torch
import torch.utils.data as data
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint, Callback

In [ ]:
en, de = l.vocabs()

In [ ]:
train, val, test = l.dataloaders((en, de), 100)

In [ ]:
model = l.LightningSeq2Seq(len(en), len(de), warmup=400, N=2, factor=1)

In [ ]:
checkpoint_path = 'tensorboard/iwslt2017'

In [ ]:
trainer = pl.Trainer(
    default_root_dir=checkpoint_path,
    gpus=0,
    max_epochs=1,
    callbacks=[
        # Save the best checkpoint based on the maximum val_acc recorded. Saves only weights and not optimizer
        ModelCheckpoint(
            save_weights_only=True, mode="max", monitor="val_loss"
        ),
        # Log learning rate every epoch
        LearningRateMonitor("step"),
        l.LogDistributions()
    ],
    fast_dev_run=False,
    log_every_n_steps=5,
    accumulate_grad_batches=2
)

In [ ]:
trainer.fit(model, train, val)